### Clase 08 - Ejemplo Adicional

In [3]:
import tensorflow as tf
import numpy as np

# 1. Componente Conexionista: La Heurística Aprendida
class LearnedHeuristic(tf.keras.Model):

  def __init__(self):
    super().__init__()
    # Red simple: toma el estado (posicion/caracteristicas) y predice un costo
    self.dense1 = tf.keras.layers.Dense(16, activation='relu')
    self.dense2 = tf.keras.layers.Dense(1) # Salida: el valor heurístico h

  def call(self, state):
    x = self.dense1(state)
    return self.dense2(x)

# 2. Componente Simbólico: El Algoritmo A*
def a_star(start, goal, neighbors_fn, cost_fn, heuristic_model):
  # Inicialización de estructuras clásicas (Open Set, G Score)
  open_set = [(0, start)]   # (f_score, nodo)
  g_score = {start: 0}

  while open_set:
    # Lógica de búsqueda A* clásica
    _, current = min(open_set, key=lambda x: x[0])
    open_set.remove((_, current))

    for neighbor in neighbors_fn(current):
      tentative_g = g_score[current] + cost_fn(current, neighbor)
      # -> PUNTO CLAVE DE INTEGRACIÓN: Usar el modelo de TF para obtener h
      state_tensor = tf.convert_to_tensor([neighbor], dtype=tf.float32)
      h = heuristic_model(state_tensor).numpy()[0][0]   # Ejecuta la NN para obtener la heurística
      f = tentative_g + h   # Usa la heurística aprendida en la funcion de evaluacion

      if neighbor not in g_score or tentative_g < g_score[neighbor]:
        g_score[neighbor] = tentative_g
        open_set.append((f, neighbor))

    return np.inf

# En resumen: A* (clásico) usa un modelo entrenable (TensorFlow) para mejorar su razonamiento.